The GPT-2 implementation is adapted from the HuggingFace library: https://huggingface.co/gpt2

In [3]:
import random
import pandas as pd
import wandb

wandb.login()

In [6]:
handle = 'realDonaldTrump' # Change handle to JoeBiden for training the model on Joe Biden's data

In [7]:
df = pd.read_csv(f'../../data/{handle}.csv')
my_tweets = df['tweet']

In [10]:
def make_dataset(dataset, epochs):
    total_text = '<|endoftext|>'
    tweets = [t for t in dataset]
    for _ in range(epochs):
        random.shuffle(tweets)
        total_text += '<|endoftext|>'.join(tweets) + '<|endoftext|>'
    return total_text

In [11]:
EPOCHS = 4

with open(f'../../data/{handle}_train.txt', 'w') as f:
    data = make_dataset(my_tweets, EPOCHS)
    f.write(data)

## Training

In [13]:
!python ../../scripts/run_language_modeling.py \
    --output_dir=output/$handle \
    --overwrite_output_dir \
    --overwrite_cache \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train --train_data_file=../../data/$handle\_train.txt \
    --logging_steps 20 \
    --per_gpu_train_batch_size 1 \
    --num_train_epochs 1

12/20/2020 13:20:18 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
12/20/2020 13:20:18 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='output/realDonaldTrump', overwrite_output_dir=True, do_train=True, do_eval=False, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=1, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec20_13-20-18_aditya-XPS-15-9570', logging_first_step=False, logging_steps=20, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_ran

 16%|█▌        | 40/258 [03:29<19:55,  5.48s/it]{'loss': 3.6806331634521485, 'learning_rate': 4.2248062015503877e-05, 'epoch': 0.15503875968992248}
                                                {'loss': 3.5538658142089843, 'learning_rate': 3.837209302325582e-05, 'epoch': 0.23255813953488372}
100%|██████████| 258/258 [22:18<00:00,  5.15s/it][INFO|trainer.py:829] 2020-12-20 13:42:44,429 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 258/258 [22:18<00:00,  5.19s/it]
[INFO|trainer.py:1222] 2020-12-20 13:42:44,436 >> Saving model checkpoint to output/realDonaldTrump
[INFO|configuration_utils.py:282] 2020-12-20 13:42:44,439 >> Configuration saved in output/realDonaldTrump/config.json
[INFO|modeling_utils.py:740] 2020-12-20 13:42:47,016 >> Model weights saved in output/realDonaldTrump/pytorch_model.bin

wandb: Waiting for W&B process to finish, PID 24316
wandb: Program ended successfully.
wandb:                                      

## Generate new tweets

In [14]:
SENTENCES = ["I think that",
             "I like",
             "I don't like",
             "I want",
             "My dream is"]

In [17]:
seed = random.randint(0, 2**32-1)
examples = []
num_return_sequences = 3

for start in SENTENCES:
    val = !python ../../run_generation.py \
        --model_type gpt2 \
        --model_name_or_path output/$handle \
        --length 160 \
        --num_return_sequences $num_return_sequences \
        --temperature 1 \
        --p 0.95 \
        --seed $seed \
        --prompt {'"<|endoftext|>' + start + '"'}
    generated = [val[-2*(k+1)] for k in range(num_return_sequences)[::-1]]
    print(f'\nStart of sentence: {start}')
    for i, g in enumerate(generated):
        g = g.replace('<|endoftext|>', '')
        print(f'* Generated #{i+1}: {g}')

['12/20/2020 13:55:50 - WARNING - __main__ -   device: cpu, n_gpu: 0, 16-bits training: False', "12/20/2020 13:55:53 - INFO - __main__ -   Namespace(device=device(type='cpu'), fp16=False, k=0, length=160, model_name_or_path='output/realDonaldTrump', model_type='gpt2', n_gpu=0, no_cuda=False, num_return_sequences=3, p=0.95, padding_text='', prefix='', prompt='<|endoftext|>I think that', repetition_penalty=1.0, seed=2314785581, stop_token=None, temperature=1.0, xlm_language='')", 'Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.', '=== GENERATED SEQUENCE 1 ===', "<|endoftext|>I think that politics is a match made in heaven. I don't want this country to look like the UK or France, I want it to look like Italy, Ireland, and the USA. There are TWO major parties.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|

['12/20/2020 13:56:28 - WARNING - __main__ -   device: cpu, n_gpu: 0, 16-bits training: False', "12/20/2020 13:56:31 - INFO - __main__ -   Namespace(device=device(type='cpu'), fp16=False, k=0, length=160, model_name_or_path='output/realDonaldTrump', model_type='gpt2', n_gpu=0, no_cuda=False, num_return_sequences=3, p=0.95, padding_text='', prefix='', prompt='<|endoftext|>My dream is', repetition_penalty=1.0, seed=2314785581, stop_token=None, temperature=1.0, xlm_language='')", 'Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.', '=== GENERATED SEQUENCE 1 ===', '<|endoftext|>My dream is that Asiana, plus Canada, see those failed states we win and they look on our state-by-state polling numbers of VOTER FORCE. We will never forgive China for allowing your vote.<|endoftext|>', '=== GENERATED SEQUENCE 2 ===', "<|endoftext|>My dream is to raise 800,000 dollars for a successful business with one of America's World Famous Listens!<|endoftext|><|endoftext|><|endoftext|><|